In [ ]:
from langchain.schema import Document
import json

# 파일 경로 
file_paths = {
    "term": "./metadata/term.json",
    "load_traffic_law": "./metadata/load_traffic_law.json",
    "modifier": "./metadata/modifier.json",
    "car_case": "./metadata/car_to_car.json",
    "precedent": "./metadata/precedent.json"
}

# 교통사고 케이스용 필드 상수
CASE_ID = "사건 ID"
CASE_TITLE = "사건 제목"
CASE_SITUATION = "사고상황"
BASE_RATIO = "기본 과실비율"
MODIFIERS = "케이스별 과실비율 조정예시"
LAW_REFERENCES = "관련 법규"
PRECEDENT = "참고 판례"
REASON = "기본 과실비율 해설"

# JSON 로드 함수
def load_json(path):
    with open(path, 'r', encoding='utf-8') as f:
        return json.load(f)

# 리스트형 JSON 변환 (term, modifier, law_meta)
def convert_list_to_documents(data_list, doc_type):
    return [
        Document(page_content=json.dumps(item, ensure_ascii=False), metadata={"type": doc_type})
        for item in data_list
    ]

def convert_car_case_documents(data_list):
    documents = []

    def safe_value(value):
        if isinstance(value, list):
            return ", ".join(map(str, value))
        elif isinstance(value, dict):
            return json.dumps(value, ensure_ascii=False)
        elif value is None:
            return ""  # null도 허용 안 되므로 빈 문자열로 처리
        else:
            return str(value)

    for item in data_list:
        if not isinstance(item, dict):
            continue

        # page_content는 원본 전체 JSON 문자열
        content = json.dumps(item, ensure_ascii=False)

        # 기본 과실비율 해설이 리스트일 수 있음 → 문자열로 병합
        reason = item.get(REASON)
        if isinstance(reason, list):
            reason = "\n".join(map(str, reason))

        metadata = {
            "type": "car_case",
            "id": safe_value(item.get(CASE_ID)),
            "title": safe_value(item.get(CASE_TITLE)),
            "situation": safe_value(item.get(CASE_SITUATION)),
            "base_ratio": safe_value(item.get(BASE_RATIO)),
            "modifiers": safe_value(item.get(MODIFIERS)),
            "load_traffic_law": safe_value(item.get(LAW_REFERENCES)),
            "precedent": safe_value(item.get(PRECEDENT)),
            "reason": safe_value(reason)
        }

        documents.append(Document(page_content=content, metadata=metadata))
    return documents

# 도로교통법 law JSON → 문서화
def convert_law_json_to_documents(data_dict):
    documents = []

    def normalize(item):
        return json.dumps(item, ensure_ascii=False) if isinstance(item, dict) else str(item)

    for law_name, content in data_dict.items():
        if isinstance(content, dict):
            for clause, text in content.items():
                lines = [normalize(x) for x in (text if isinstance(text, list) else [text])]
                full_text = f"{law_name} {clause}\n" + "\n".join(lines)
                documents.append(Document(page_content=full_text, metadata={"type": "load_traffic_law"}))
        else:
            lines = [normalize(x) for x in (content if isinstance(content, list) else [content])]
            full_text = f"{law_name}\n" + "\n".join(lines)
            documents.append(Document(page_content=full_text, metadata={"type": "load_traffic_law"}))
    
    return documents


# 문서화 실행
term_docs = convert_list_to_documents(load_json(file_paths["term"]), "term")
modifier_docs = convert_list_to_documents(load_json(file_paths["modifier"]), "modifier")
precedent_docs = convert_list_to_documents(load_json(file_paths["precedent"]), "precedent")
car_case_docs = convert_car_case_documents(load_json(file_paths["car_case"]))
load_traffic_law_docs = convert_law_json_to_documents(load_json(file_paths["load_traffic_law"]))


# 전체 문서 리스트
all_docs = term_docs + modifier_docs + car_case_docs + precedent_docs + load_traffic_law_docs

In [ ]:
from langchain.vectorstores import Chroma  # persist 지원
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

# 1. 청크 크기 조정 (500~1000 권장)
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=100,
    length_function=len,
    separators=["\n\n", "\n", "(?<=\\. )", " ", ""],
    is_separator_regex=True,
)

# 2. 문서 분할
all_splits = text_splitter.split_documents(all_docs)

embedding_model = OpenAIEmbeddings()

# 4. Chroma DB에 배치 처리로 저장
batch_size = 100  # 한 번에 처리할 청크 수
vectorstore = Chroma.from_documents(
    documents=all_splits[:batch_size],  # 첫 배치
    embedding=embedding_model,
    persist_directory="./vectordb"
)

# 남은 청크를 순차적으로 추가 
for i in range(0, len(all_splits), batch_size):
    try:
        batch = all_splits[i:i+batch_size]
        vectorstore.add_documents(batch)
        vectorstore.persist()  # 매 배치 후 즉시 저장
    except Exception as e:
        print(f"배치 {i}~{i+batch_size} 저장 실패: {e}")

vectorstore.persist()


In [ ]:
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain_openai import ChatOpenAI
from langchain.vectorstores import Chroma

# 1. LLM 설정
model = ChatOpenAI(model='gpt-4o', temperature=0.5)

# 2. 프롬프트 템플릿
prompt = PromptTemplate(
    template="""
다음 '사고 상황 설명'에 대해 '문서'의 내용만 참고하여 과실 비율 및 법적 판단을 생성해 주세요.

사고 상황: {question}

문서 내용: {context}

답변 형식:
1. 과실비율: A차량 xx% vs B차량 xx%
2. 사고 원인 및 판단 근거:
   - [사고의 전개, 각 차량의 행위, 과실 요소 등을 구체적으로 설명]
3. 관련 법률 조항:
   - [예: 도로교통법 제10조 제2항, 제27조 제1항 등]
4. 참고 판례:
   - [예: 대법원 2023다12345, 서울중앙지법 2022가단123456 등]

조건:
- 반드시 문서 내용만 참고해 판단하세요.
- 법률 조항과 판례가 명시되어 있지 않으면 유사하거나 추정 가능한 근거를 제시해도 됩니다.
- 전체 답변은 포맷에 맞게 간결하고 전문적으로 작성하세요.
""",
    input_variables=["question", "context"]
)

# 3. 리트리버 설정
retriever = vectorstore.as_retriever(search_kwargs={"k": 5})

# 4. QA 체인 재구성 (수정 부분)
qa_chain = RetrievalQA.from_chain_type(
    llm=model,
    retriever=retriever,
    chain_type="stuff",
    chain_type_kwargs={"prompt": prompt},
    return_source_documents=True
)

# 5. 실제 질의 실행
query = "B차량이 비보호좌회전 표지가 없는 교차로에서 녹색등(좌회전 화살표 없음)에 좌회전 시도, 직진하는 A차량과 충돌한 사고가 발생했어. 과실비율이 어떻게 돼??"

res = qa_chain.invoke({"query": query})

# 6. 출력
print("✅ 답변:\n", res["result"])


In [ ]:
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA

detail_prompt = PromptTemplate(
    template="""
아래 문서 내용을 바탕으로 사용자가 물어본 용어나 법률 조항, 판례에 대해 정확하고 간결하게 설명해 주세요.

질문: {question}

문서: {context}

답변 형식:
- 용어/조항 정의: [정확한 설명]
- 출처가 명시된 경우: 관련 법률/조문 번호/판례명을 반드시 포함

답변:
""",
    input_variables=["question", "context"]
)

detail_chain = RetrievalQA.from_chain_type(
    llm=model,
    retriever=retriever,
    chain_type="stuff",
    chain_type_kwargs={"prompt": detail_prompt}
)

# run은 question 키를 기준으로 실행됨!
res2 = detail_chain.run("보행자우선도로가 뭔지 설명해줘")

print("✅ 기능② 답변:\n", res2)


In [ ]:
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain_openai import ChatOpenAI

# LLM 설정
model = ChatOpenAI(model='gpt-4o', temperature=0.3)

# 기능 1번. 사고 상황 판단 프롬프트
prompt = PromptTemplate(
    template="""
너는 교통사고 과실 판단 전문가야.
다음 '사고 상황'을 보고, 아래 문서 내용을 기반으로 과실비율을 계산해줘.

사고 상황:
{question}

문서:
{context}

출력 형식:
1. 과실비율: A차량 xx% vs B차량 xx%
2. 판단 근거 요약
3. 적용 법률
4. 참고 판례

조건:
- 반드시 문서 내용 기반으로 판단해
- 문서에 없다면 판단 보류하거나 유사 사례로 근거 설명해
""",
    input_variables=["question", "context"]
)


# 기능 2번. 용어/법률 설명 프롬프트
detail_prompt = PromptTemplate(
    template="""
아래 문서 내용을 바탕으로 사용자가 물어본 용어나 법률 조항, 판례에 대해 정확하고 간결하게 설명해 주세요.

질문: {question}

문서: {context}

답변 형식:
- 용어/조항 정의: [정확한 설명]
- 출처가 명시된 경우: 관련 법률/조문 번호/판례명을 반드시 포함

답변:
""",
    input_variables=["question", "context"]
)

# 리트리버 설정
retriever = vectorstore.as_retriever(search_kwargs={"k": 5})

# 사고 상황 판단 체인
accident_chain = RetrievalQA.from_chain_type(
    llm=model,
    retriever=retriever,
    chain_type="stuff",
    chain_type_kwargs={"prompt": prompt}
)

# 설명용 체인
detail_chain = RetrievalQA.from_chain_type(
    llm=model,
    retriever=retriever,
    chain_type="stuff",
    chain_type_kwargs={"prompt": detail_prompt}
)

def classify_query(user_input: str) -> str:
    accident_keywords = ["충돌", "사고", "과실", "A차량", "B차량", "신호위반", "부딪힘", "진입", "교차로"]
    explanation_keywords = ["무엇", "뭐야", "정의", "설명", "판례", "조항", "법원", "법", "도로교통법" ,"무슨 뜻", "의미", "용어", "이란"]

    # 기능② 설명 우선 처리
    if any(word in user_input for word in explanation_keywords):
        return "detail"
    if any(word in user_input for word in accident_keywords):
        return "accident"
    return "detail"


# 통합 실행 함수
def run_query(user_input: str):
    category = classify_query(user_input)
    if category == "accident":
        result = accident_chain.run(user_input)
        print("✅ 사고 상황 판단 결과:\n", result)
    else:
        result = detail_chain.run(user_input)
        print("📘 용어/법률 설명 결과:\n", result)

### 테스트

In [ ]:
run_query("A차량이 적색신호에 직진 중이고, B차량은 좌회전 신호 없이 녹색등에서 좌회전 중 충돌한 사고야. 과실비율은?")

In [ ]:
run_query("비보호 좌회전이 뭔지 설명해줘")

In [ ]:
run_query("대법원 92도2077 판결문 내용을 알려줘")

In [ ]:
run_query("서울고등법원 2002나57692 판결문 내용을 알려줘")